<a href="https://colab.research.google.com/github/Gaurav-Ramachandra/Sem5-BIS_Lab/blob/main/Expt3%207-11%3A%20%20ACO%20for%20TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

class AntColony:
    def __init__(self, distances, num_ants, num_iterations, decay, alpha=1, beta=2, Q=1):
        self.distances = distances                  # Distance matrix
        self.pheromone = np.ones(self.distances.shape) / len(distances)  # Initial pheromone levels
        self.num_ants = num_ants                    # Number of ants
        self.num_iterations = num_iterations        # Number of iterations
        self.decay = decay                          # Pheromone decay rate
        self.alpha = alpha                          # Influence of pheromone
        self.beta = beta                            # Influence of distance
        self.Q = Q                                  # Pheromone constant

    def run(self):
        best_path = None
        best_length = float("inf")

        for _ in range(self.num_iterations):
            all_paths = self.construct_solutions()  # Construct solutions for all ants
            self.spread_pheromone(all_paths)        # Update pheromone levels
            shortest_path, shortest_length = min(all_paths, key=lambda x: x[1])

            # Update global best path
            if shortest_length < best_length:
                best_path = shortest_path
                best_length = shortest_length

            # Evaporate some pheromones
            self.pheromone *= (1 - self.decay)

        return best_path, best_length

    def construct_solutions(self):
        all_paths = []
        for _ in range(self.num_ants):
            path = self.construct_path()
            path_length = self.calculate_path_length(path)
            all_paths.append((path, path_length))
        return all_paths

    def construct_path(self):
        path = []
        visited = set()
        current_city = random.randint(0, len(self.distances) - 1)
        path.append(current_city)
        visited.add(current_city)

        while len(visited) < len(self.distances):
            next_city = self.choose_next_city(current_city, visited)
            path.append(next_city)
            visited.add(next_city)
            current_city = next_city

        path.append(path[0])  # Return to the starting city to complete the tour
        return path

    def choose_next_city(self, current_city, visited):
        probabilities = []
        for next_city in range(len(self.distances)):
            if next_city not in visited:
                pheromone = self.pheromone[current_city][next_city] ** self.alpha
                visibility = (1 / self.distances[current_city][next_city]) ** self.beta
                probabilities.append(pheromone * visibility)
            else:
                probabilities.append(0)

        total = sum(probabilities)
        if total == 0:
            return random.choice([city for city in range(len(self.distances)) if city not in visited])

        probabilities = [prob / total for prob in probabilities]
        return np.random.choice(range(len(self.distances)), p=probabilities)

    def calculate_path_length(self, path):
        length = 0
        for i in range(len(path) - 1):
            length += self.distances[path[i]][path[i + 1]]
        return length

    def spread_pheromone(self, all_paths):
        for path, path_length in all_paths:
            for i in range(len(path) - 1):
                from_city = path[i]
                to_city = path[i + 1]
                self.pheromone[from_city][to_city] += self.Q / path_length
                self.pheromone[to_city][from_city] += self.Q / path_length

# Example Usage
if __name__ == "__main__":
    # Distance matrix representing distances between 5 cities
    distances = np.array([
        [0, 2, 2, 5, 7],
        [2, 0, 4, 8, 2],
        [2, 4, 0, 1, 3],
        [5, 8, 1, 0, 2],
        [7, 2, 3, 2, 0]
    ])

    # Initialize parameters
    num_ants = 10
    num_iterations = 100
    decay = 0.5
    alpha = 1
    beta = 2
    Q = 1

    # Run ACO
    aco = AntColony(distances, num_ants, num_iterations, decay, alpha, beta, Q)
    best_path, best_length = aco.run()

    # Output results
    print("Best path found:", best_path)
    print("Path length:", best_length)


Best path found: [2, 0, 1, 4, 3, 2]
Path length: 9
